In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import matplotlib as mpl
mpl.rcParams["lines.linewidth"] = 0.96
%matplotlib qt

In [3]:
plt.style.use('ggplot')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("talk")

# Single-values

In [4]:
data_path = "/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Data/paper/"

df_lab = pd.read_csv(data_path + 'features_LAB.csv')
df_sleep = pd.read_csv(data_path + 'features_SLEEP.csv')
df_wake = pd.read_csv(data_path + 'features_WAKE.csv')

## T-test

In [8]:
from scipy.stats import ttest_ind

# Define a function to perform t-tests on each feature
def perform_ttests(df):
    features = df.columns[1:-1]  # Exclude the ID and SBP columns
    results = {}
    for feature in features:
        high_bp = df[df['SBP'] == 'High BP'][feature]
        low_bp = df[df['SBP'] == 'Low BP'][feature]
        t_stat, p_value = ttest_ind(high_bp, low_bp, equal_var=False)
        results[feature] = {'t_stat': t_stat, 'p_value': p_value}
    return results

# Perform t-tests on each dataframe
results_lab = perform_ttests(df_lab)
results_sleep = perform_ttests(df_sleep)
results_wake = perform_ttests(df_wake)

In [9]:
# Function to plot the results
def plot_significant_features(results, title):
    # Convert results to DataFrame
    df_results = pd.DataFrame(results).T
    # Add a column for significance
    df_results['Significant'] = df_results['p_value'] < 0.05
    # Sort by p-value
    df_results = df_results.sort_values(by='p_value')
    
    plt.figure(figsize=(10, 8))
    sns.barplot(x=df_results.index, y='p_value', hue='Significant', data=df_results, dodge=False, palette='coolwarm')
    plt.axhline(0.05, color='red', linestyle='--')
    plt.xticks(rotation=90)
    plt.title(title)
    plt.ylabel('p-value')
    plt.xlabel('Features')
    plt.legend(title='Significant', frameon=True, fancybox=True, shadow=True)
    plt.tight_layout()
    plt.show()

# Plot the results for each dataframe
plot_significant_features(results_lab, 'Lab Results - PPG Features Significance')
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Figures/significant_features_highVSlow_lab.png", dpi = 300, bbox_inches = 'tight')
plot_significant_features(results_sleep, 'Sleep Results - PPG Features Significance')
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Figures/significant_features_highVSlow_sleep.png", dpi = 300, bbox_inches = 'tight')
plot_significant_features(results_wake, 'Wake Results - PPG Features Significance')
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Figures/significant_features_highVSlow_wake.png", dpi = 300, bbox_inches = 'tight')

## Boxplot

In [51]:
feature = "Tsp"

plt.figure(figsize=(12, 7))
plt.subplot(1, 3, 1)
sns.boxplot(data = df_lab, x = "SBP", y = feature, hue = "SBP", showfliers = True)
plt.title("Lab")
plt.subplot(1, 3, 2, sharey = plt.subplot(1, 3, 1))
sns.boxplot(data = df_wake, x = "SBP", hue = "SBP", y = feature, showfliers = True)
plt.title("Wake")
plt.subplot(1, 3, 3, sharey = plt.subplot(1, 3, 1))
sns.boxplot(data = df_sleep, x = "SBP", hue = "SBP", y = feature, showfliers = False)
plt.title("Sleep")
plt.tight_layout()

# 24-hours

In [5]:
features_hourly = pd.read_csv(data_path + 'features_24h_hourly.csv')
# Create a new column for ordered hours (12pm to 12pm)
features_hourly["hour"] = features_hourly["hour"].values + 2
features_hourly['hour_ordered'] = features_hourly['hour'].apply(lambda x: x if x >= 12 else x + 24)

# Separate the data based on SBP (hypertensive vs. non-hypertensive)
hypertensive = features_hourly[features_hourly['SBP'] == 1]
non_hypertensive = features_hourly[features_hourly['SBP'] == 0]

## Plot

In [6]:
from statsmodels.stats.multitest import multipletests

In [7]:
feature = "Tsp"
feature_name = "Systolic Peak Time"
# Calculate the mean and standard error of Tsp for each hour of the day in the new order for both groups
mean_hyper = hypertensive.groupby('hour_ordered')[feature].mean().sort_index() # this takes the mean of the values for each hour of the day
sem_hyper = hypertensive.groupby('hour_ordered')[feature].sem().sort_index()
mean_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].mean().sort_index()
sem_non_hyper = non_hypertensive.groupby('hour_ordered')[feature].sem().sort_index()

median_lab_hyper = df_lab[df_lab['SBP'] == 'High BP'][feature].median()
sem_lab_hyper = df_lab[df_lab['SBP'] == 'High BP'][feature].sem()
median_lab_non_hyper = df_lab[df_lab['SBP'] == 'Low BP'][feature].median()
sem_lab_non_hyper = df_lab[df_lab['SBP'] == 'Low BP'][feature].sem()

# Perform t-tests for each hour of the day
p_values = []
t_stats = []
for hour in sorted(features_hourly['hour_ordered'].unique()):
    hyper_values = hypertensive[hypertensive['hour_ordered'] == hour][feature]
    non_hyper_values = non_hypertensive[non_hypertensive['hour_ordered'] == hour][feature]
    hyper_values = hyper_values.dropna()
    non_hyper_values = non_hyper_values.dropna()
    t_stat, p_val = ttest_ind(hyper_values, non_hyper_values, equal_var=False)
    p_values.append(p_val)
    t_stats.append(t_stat)

adjusted_p_values = multipletests(p_values, alpha=0.05, method='bonferroni')[1]

# Adjust the x-axis labels to match the new order
ordered_hours_labels = [f"{hour % 24}:00" for hour in mean_hyper.index]
# plt.figure(figsize=(12, 7))
# plt.errorbar(range(len(mean_hyper)), mean_hyper.values, yerr=sem_hyper.values, fmt='-o', capsize=5, label='High BP')
# plt.errorbar(range(len(mean_non_hyper)), mean_non_hyper.values, yerr=sem_non_hyper.values, fmt='-o', capsize=5, label='Low BP')
# # plt.axhline(y=median_lab_hyper, linestyle='--', linewidth=0.9, label='LAB High BP')
# # plt.axhline(y=median_lab_non_hyper, linestyle='--', color = "b", linewidth=0.9, label='LAB Low BP')
# plt.xlabel('Hour of the Day')
# plt.ylabel(feature_name + ' (samples)')
# plt.title(f'{feature_name} per Hour of the Day (12pm to 12pm)')
# plt.grid(True)
# plt.xticks(range(len(mean_hyper)), ordered_hours_labels, rotation=45)
# plt.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
# plt.show()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 9), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

# Top plot: feature values with error bars
ax1.errorbar(range(len(mean_hyper)), mean_hyper.values, yerr=sem_hyper.values, fmt='-o', capsize=5, label='High BP')
ax1.errorbar(range(len(mean_non_hyper)), mean_non_hyper.values, yerr=sem_non_hyper.values, fmt='-o', capsize=5, label='Low BP')
ax1.set_ylabel(feature_name + ' (samples)')
ax1.set_title(f'{feature_name} over 24 hours')
ax1.grid(True)
ax1.set_xticks([])
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)

# Bottom plot: p-values
#ax2.bar(range(len(adjusted_p_values)), adjusted_p_values, color = 'lightblue')
ax2.bar(range(len(p_values)), p_values, color = 'lightblue')
ax2.axhline(y=0.01, color='r', linestyle='--', label='p=0.01', linewidth=1.2)
ax2.set_xlabel('Hour of the Day')
ax2.set_ylabel('p-value')
ax2.grid(True)
ax2.set_xticks(range(len(p_values)))
ax2.set_xticklabels(ordered_hours_labels, rotation=45)
ax2.set_ylim(0, 0.1)
ax2.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, loc = "upper right")
# Add stars above significant bars
for i, p_value in enumerate(p_values):
    if p_value < 0.01:
        ax2.text(i,  0.05, '*', ha='center', va='bottom', fontsize=29, color='black')

plt.tight_layout()

plt.tight_layout()
plt.show()

NameError: name 'ttest_ind' is not defined

In [66]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Figures/Tsp_hourly_with_stats.png", dpi = 300, bbox_inches = 'tight')

In [67]:
plt.close('all')

## PPG average waveform

In [68]:
import glob
data_path = '/Volumes/Untitled/HOM3ostasis/CinC/Data_24hPulses/mod_newQuality/'
HOM_path = '/Volumes/Untitled/Hom3ostasis/HOM_mod/HOM_mod/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])

ppg_diag_df_all = pd.DataFrame()

for i, sub_ID in enumerate(sub_names): 

    # fiducials_diag = pd.read_csv(data_path + sub_ID + '/fiducials_diag_ALL.csv')
    # features_diag = pd.read_csv(data_path + sub_ID + '/features_diag_ALL.csv')
    # fiducials_diag['sub_ID'] = sub_ID
    # features_diag['sub_ID'] = sub_ID

    fiducials_diag = pd.read_csv(data_path + sub_ID + '/fiducials_basic_ALL.csv')
    features_diag = pd.read_csv(data_path + sub_ID + '/features_basic_ALL.csv')
    fiducials_diag['sub_ID'] = sub_ID
    features_diag['sub_ID'] = sub_ID
    
    fiducials_diag["sleep"] = features_diag["sleep"].values

    SBP = df_lab[df_lab["Unnamed: 0"] == sub_ID]["SBP"].values[0]

    onsets = fiducials_diag["onset"].values
    offsets = fiducials_diag["offset"].values

    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]

    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
    ppg = ppg_data[2:].to_numpy().squeeze()
    t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')
    ppg_df = pd.DataFrame(ppg, index=t_ppg, columns=["ppg"])

    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1

    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ppg = ppg_df["ppg"].to_numpy()
    ppg_pulses = [ppg[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]
    ppg = np.concatenate(ppg_pulses_interp)

    ppg_diag_list = []
    for i in range(len(onsets)):
        ppg_diag_pulse = ppg[onsets[i]:offsets[i]]
        ppg_diag_list.append(ppg_diag_pulse)

    ppg_diag_df = pd.DataFrame(ppg_diag_list)
    ppg_diag_df["sleep"] = fiducials_diag["sleep"].values
    ppg_diag_df["SBP"] = SBP
    ppg_diag_df["sub_ID"] = sub_ID

    ppg_diag_df_all = pd.concat([ppg_diag_df_all, ppg_diag_df])


In [69]:
ppg_diag_df_all_wake = ppg_diag_df_all[ppg_diag_df_all["sleep"] == 0]
ppg_diag_df_all_sleep = ppg_diag_df_all[ppg_diag_df_all["sleep"] == 1]

ppg_diag_df_all_wake_mean = ppg_diag_df_all_wake.drop(columns = ["sub_ID", "sleep"]).groupby("SBP").mean().T
ppg_diag_df_all_sleep_mean = ppg_diag_df_all_sleep.drop(columns = ["sub_ID", "sleep"]).groupby("SBP").mean().T


In [7]:
ppg_diag_df_all_wake_mean.to_csv(data_path + 'ppg_diag_df_all_wake_mean.csv')
ppg_diag_df_all_sleep_mean.to_csv(data_path + 'ppg_diag_df_all_sleep_mean.csv')

NameError: name 'ppg_diag_df_all_wake_mean' is not defined

In [8]:
data_path = '/Volumes/Untitled/HOM3ostasis/CinC/Data_24hPulses/mod_newQuality/'

ppg_diag_df_all_wake_mean = pd.read_csv(data_path + 'ppg_diag_df_all_wake_mean.csv', index_col = 0)
ppg_diag_df_all_sleep_mean = pd.read_csv(data_path + 'ppg_diag_df_all_sleep_mean.csv', index_col = 0)

### High vs Low BP

In [9]:
Tsp_sleep_high = np.argmax(ppg_diag_df_all_sleep_mean["High BP"].values)
Tsp_sleep_low = np.argmax(ppg_diag_df_all_sleep_mean["Low BP"].values)

plt.figure(figsize=(9, 7))
plt.plot(ppg_diag_df_all_sleep_mean["High BP"], linewidth = 1.9, label = "High BP")
plt.plot(ppg_diag_df_all_sleep_mean["Low BP"], linewidth = 1.9, label = "Low BP")
plt.scatter(Tsp_sleep_high, ppg_diag_df_all_sleep_mean["High BP"].iloc[Tsp_sleep_high], color = "r", s = 100)
plt.scatter(Tsp_sleep_low, ppg_diag_df_all_sleep_mean["Low BP"].iloc[Tsp_sleep_low], color = "b", s = 100)
# plt.axvline(x=Tsp_sleep_high, linestyle='--', linewidth=1.4)
# plt.axvline(x=Tsp_sleep_low, linestyle='--', color = "b", linewidth=1.4)
plt.vlines(x=Tsp_sleep_high, ymin = ppg_diag_df_all_sleep_mean["High BP"].iloc[Tsp_sleep_high], ymax = 1000, color = "r", linestyle='--', linewidth=1.4)
plt.vlines(x=Tsp_sleep_low, ymin = ppg_diag_df_all_sleep_mean["Low BP"].iloc[Tsp_sleep_low], ymax = 1000, color = "b", linestyle='--', linewidth=1.4)
plt.text(Tsp_sleep_high+1, 172, f'{Tsp_sleep_high}', fontsize=18, ha='center', va='bottom', color = "r")
plt.text(Tsp_sleep_low-1, 172, f'{Tsp_sleep_low}', fontsize=18, ha='center', va='bottom', color = "b")
plt.ylim(-5, 170)
plt.grid()
plt.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
plt.title("Sleep", fontsize = 24)

Text(0.5, 1.0, 'Sleep')

### Wake vs Sleep

In [12]:
plt.figure(figsize=(9, 7))
plt.plot(ppg_diag_df_all_wake_mean.mean(axis = 1), linewidth = 1.9, label = "Wake")
plt.plot(ppg_diag_df_all_sleep_mean.mean(axis = 1), linewidth = 1.9, label = "Sleep")
plt.ylim(-5, 170)
plt.grid()
plt.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
plt.title("Wake vs Sleep", fontsize = 24)

Text(0.5, 1.0, 'Wake vs Sleep')

In [136]:
ppg_diag_df_all.drop(columns = "sub_ID").groupby(["sleep", "SBP"]).mean().T.plot(kind='line', figsize=(12, 7))

<Axes: >

In [82]:
# from list to np.array
ppg_diag_array = np.array(ppg_diag_list)
ppg_diag_mean = ppg_diag_array.mean(axis = 0)

plt.figure()
plt.plot(ppg_diag_mean)

In [66]:
plt.figure()
plt.plot(ppg)

In [87]:
df_lab["SBP"]

0     High BP
1     High BP
2      Low BP
3      Low BP
4      Low BP
5     High BP
6      Low BP
7     High BP
8     High BP
9     High BP
10     Low BP
11     Low BP
12     Low BP
13     Low BP
14     Low BP
15     Low BP
16     Low BP
17     Low BP
18    High BP
19     Low BP
20    High BP
21    High BP
22     Low BP
23     Low BP
24     Low BP
25     Low BP
26     Low BP
27    High BP
28    High BP
29     Low BP
30     Low BP
31     Low BP
32     Low BP
33     Low BP
34     Low BP
35    High BP
36     Low BP
37    High BP
38     Low BP
Name: SBP, dtype: object

In [91]:
ppg_diag_mean_all["SBP"] = df_lab["SBP"].values

In [92]:
ppg_diag_mean_all

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,SBP
10_AC,-43.853980,-43.355727,-42.762074,-41.434639,-39.817404,-37.676326,-35.205322,-32.369870,-29.281071,-25.931655,...,-30.243654,-32.159888,-34.078889,-35.977423,-37.816537,-39.509224,-41.092473,-42.295786,-43.454210,High BP
11_EC,-50.494772,-49.832669,-48.458824,-46.265955,-43.169036,-39.385912,-35.125503,-30.491474,-25.666146,-20.792165,...,-33.814582,-35.796031,-37.922832,-40.179461,-42.506298,-44.832286,-46.978888,-48.813685,-50.316667,High BP
12_FN,-33.096298,-32.527943,-31.795647,-30.091988,-27.970000,-25.111208,-21.751943,-17.947303,-13.803298,-9.488696,...,-22.410012,-23.538265,-24.772656,-26.092719,-27.472157,-28.860551,-30.226054,-31.370066,-32.480167,Low BP
13_MS,-105.116361,-103.407532,-101.387660,-95.898410,-89.750202,-80.604620,-70.755649,-58.934957,-46.693567,-33.668304,...,-75.485794,-77.475916,-79.688995,-82.638079,-85.764528,-89.371378,-93.003664,-96.262865,-99.455131,Low BP
14_AD,-39.338416,-38.771686,-37.963394,-36.086562,-33.767648,-30.559546,-26.890674,-22.649968,-18.035136,-13.163434,...,-30.217987,-31.016768,-31.934530,-32.965778,-34.084829,-35.273962,-36.463180,-37.512957,-38.513705,Low BP
15_ER,-85.620589,-84.416796,-82.304382,-78.553021,-73.230922,-66.697897,-59.141940,-50.772686,-41.941834,-32.891438,...,-62.288336,-65.532560,-68.964489,-72.592013,-76.332634,-80.062449,-83.612244,-86.615963,-89.299671,High BP
16_PD,-87.486429,-86.131273,-83.347321,-78.647281,-71.834850,-63.341345,-53.588766,-42.903440,-31.786018,-20.654318,...,-62.156236,-64.866475,-67.844141,-71.118963,-74.645647,-78.346574,-81.997257,-85.218105,-88.060801,Low BP
17_EG,-69.681246,-68.597766,-67.013616,-64.194801,-60.259526,-55.515419,-49.840623,-43.629731,-36.900026,-29.898287,...,-47.592841,-50.364408,-53.142102,-55.910770,-58.620986,-61.228353,-63.600926,-65.560809,-67.343383,High BP
18_MV,-41.172613,-40.697760,-39.951916,-38.563265,-36.662649,-34.326000,-31.565774,-28.479026,-25.138861,-21.609041,...,-28.627394,-30.364032,-32.154930,-33.974102,-35.770158,-37.478163,-39.039813,-40.286325,-41.391736,High BP
19_FI,-68.001661,-67.210502,-65.667560,-63.133210,-59.671761,-55.454954,-50.579033,-45.212749,-39.439769,-33.350230,...,-46.662099,-49.550211,-52.616252,-55.820787,-59.082971,-62.252442,-65.131091,-67.521158,-69.459659,High BP


In [94]:
ppg_diag_mean_all["SBP"]

10_AC    High BP
11_EC    High BP
12_FN     Low BP
13_MS     Low BP
14_AD     Low BP
15_ER    High BP
16_PD     Low BP
17_EG    High BP
18_MV    High BP
19_FI    High BP
1_ID      Low BP
20_AC     Low BP
21_CK     Low BP
22_CM     Low BP
23_PP     Low BP
24_SM     Low BP
25_LP     Low BP
26_CB     Low BP
27_CM    High BP
28_GM     Low BP
29_LT    High BP
30_LN    High BP
31_FB     Low BP
32_CD     Low BP
33_NM     Low BP
34_AS     Low BP
35_AA     Low BP
36_GS    High BP
37_IC    High BP
38_GC     Low BP
39_NB     Low BP
3_JF      Low BP
40_SQ     Low BP
4_MF      Low BP
5_PD      Low BP
6_SS     High BP
7_GM      Low BP
8_MB     High BP
9_BC      Low BP
Name: SBP, dtype: object

In [96]:
ppg_diag_mean_all[ppg_diag_mean_all["SBP"].values == "Low BP"]

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,SBP
12_FN,-33.096298,-32.527943,-31.795647,-30.091988,-27.970000,-25.111208,-21.751943,-17.947303,-13.803298,-9.488696,...,-22.410012,-23.538265,-24.772656,-26.092719,-27.472157,-28.860551,-30.226054,-31.370066,-32.480167,Low BP
13_MS,-105.116361,-103.407532,-101.387660,-95.898410,-89.750202,-80.604620,-70.755649,-58.934957,-46.693567,-33.668304,...,-75.485794,-77.475916,-79.688995,-82.638079,-85.764528,-89.371378,-93.003664,-96.262865,-99.455131,Low BP
14_AD,-39.338416,-38.771686,-37.963394,-36.086562,-33.767648,-30.559546,-26.890674,-22.649968,-18.035136,-13.163434,...,-30.217987,-31.016768,-31.934530,-32.965778,-34.084829,-35.273962,-36.463180,-37.512957,-38.513705,Low BP
16_PD,-87.486429,-86.131273,-83.347321,-78.647281,-71.834850,-63.341345,-53.588766,-42.903440,-31.786018,-20.654318,...,-62.156236,-64.866475,-67.844141,-71.118963,-74.645647,-78.346574,-81.997257,-85.218105,-88.060801,Low BP
1_ID,-73.949144,-72.781804,-70.714849,-66.906100,-61.433248,-54.627352,-46.552642,-37.568186,-28.047007,-18.298432,...,-50.121817,-52.626373,-55.311767,-58.185186,-61.215336,-64.275228,-67.293071,-69.871599,-72.239859,Low BP
20_AC,-60.105588,-58.998277,-57.754747,-54.787609,-51.347460,-46.779028,-41.707589,-36.144144,-30.287544,-24.394368,...,-37.880018,-40.441464,-43.200475,-46.201161,-49.243469,-52.296014,-55.245430,-57.634987,-59.967495,Low BP
21_CK,-43.114839,-42.536449,-41.609235,-39.791052,-37.438790,-34.383081,-30.907133,-27.045998,-22.937691,-18.722368,...,-29.699677,-31.268939,-32.961068,-34.720583,-36.532211,-38.295012,-39.997699,-41.372130,-42.626422,Low BP
22_CM,-112.151185,-110.194004,-105.552822,-98.090100,-87.813369,-75.166539,-61.016649,-46.047615,-30.916401,-16.202693,...,-75.249734,-78.674720,-82.502635,-86.824999,-91.675519,-96.936284,-102.208724,-107.022571,-111.032027,Low BP
23_PP,-72.819405,-71.961762,-70.486113,-67.552220,-63.236279,-57.709783,-50.797492,-42.878314,-34.097745,-24.777165,...,-57.748806,-59.519860,-61.276807,-63.075238,-64.918099,-66.773352,-68.633665,-70.257853,-71.793575,Low BP
24_SM,-157.906542,-155.325861,-150.227267,-141.222685,-128.024511,-111.195746,-91.674560,-69.730084,-46.462223,-22.700467,...,-112.539816,-116.724795,-121.382166,-126.569448,-132.202831,-138.187543,-144.245969,-149.676096,-154.615920,Low BP


In [101]:
ppg_diag_mean_low_bp = ppg_diag_mean_all[ppg_diag_mean_all["SBP"].values == "Low BP"].drop(columns = "SBP").mean()
ppg_diag_mean_high_bp = ppg_diag_mean_all[ppg_diag_mean_all["SBP"].values == "High BP"].drop(columns = "SBP").mean()

plt.figure()
plt.plot(ppg_diag_mean_low_bp)
plt.plot(ppg_diag_mean_high_bp)
plt.legend(["Low BP", "High BP"])
